<a href="https://colab.research.google.com/github/caroheymes/aws-upload-demo/blob/master/aws_upload_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boto3 upload
# Get Cities of The World Quality of Life Data

We will try to get scoring information about the quality of life for several cities around the world. 🌍

For this exercise, we will be using the following API:

Teleport

We will also need to extract data from wikipedia.com that will give us a random cities around the world to get a scoring.

Then we will store the data we got into an S3 Bucket!

Quite a project, right? 🥰🥰🥰🥰

So let's go 💪💪💪

In [61]:
!pip install boto3

In [3]:
import os
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [62]:
import pandas as pd

creds = pd.read_csv('/content/Caroline_accessKeys.csv')
print(creds.columns)
print(creds.columns)
aws_access_key_id = creds['Access key ID'][0]
aws_secret_access_key = creds['Secret access key'][0]

Index(['Access key ID', 'Secret access key'], dtype='object')
Index(['Access key ID', 'Secret access key'], dtype='object')


In [ ]:
import boto3
session = boto3.Session(aws_access_key_id=aws_access_key_id, 
                        aws_secret_access_key=aws_secret_access_key)

In [ ]:
s3 = session.resource("s3")

In [17]:
s3.create_bucket(Bucket="currentbuckettoday")

s3.Bucket(name='currentbuckettoday')

**Getting data on Paris**

In [18]:
import requests

In [19]:
r = requests.get('https://api.teleport.org/api/cities/?search=Paris').json()

In [29]:
url = r['_embedded']['city:search-results'][0]['_links']['city:item']['href']
url

'https://api.teleport.org/api/cities/geonameid:2988507/'

In [34]:
city_requests = requests.get(url).json()
city_requests

{'_links': {'city:admin1_division': {'href': 'https://api.teleport.org/api/countries/iso_alpha2:FR/admin1_divisions/geonames:11/',
   'name': 'Île-de-France'},
  'city:alternate-names': {'href': 'https://api.teleport.org/api/cities/geonameid:2988507/alternate_names/'},
  'city:country': {'href': 'https://api.teleport.org/api/countries/iso_alpha2:FR/',
   'name': 'France'},
  'city:timezone': {'href': 'https://api.teleport.org/api/timezones/iana:Europe%2FParis/',
   'name': 'Europe/Paris'},
  'city:urban_area': {'href': 'https://api.teleport.org/api/urban_areas/slug:paris/',
   'name': 'Paris'},
  'curies': [{'href': 'https://developers.teleport.org/api/resources/Location/#!/relations/{rel}/',
    'name': 'location',
    'templated': True},
   {'href': 'https://developers.teleport.org/api/resources/City/#!/relations/{rel}/',
    'name': 'city',
    'templated': True},
   {'href': 'https://developers.teleport.org/api/resources/UrbanArea/#!/relations/{rel}/',
    'name': 'ua',
    'templa

In [48]:
pd.DataFrame(requests.get('https://api.teleport.org/api/urban_areas/slug:paris/scores/').json()['categories'])[['name',	'score_out_of_10']].to_csv('paris.csv')

In [51]:
s3.Bucket('currentbuckettoday').upload_file('/content/paris.csv', Key = 'paris_file.csv')

**data collection**

In [52]:
!pip install scrapy -q

     |████████████████████████████████| 254 kB 8.6 MB/s 
     |████████████████████████████████| 65 kB 4.3 MB/s 
     |████████████████████████████████| 47 kB 5.7 MB/s 
     |████████████████████████████████| 251 kB 53.6 MB/s 
     |████████████████████████████████| 55 kB 4.3 MB/s 
     |████████████████████████████████| 3.1 MB 50.3 MB/s 
     |████████████████████████████████| 3.6 MB 50.4 MB/s 
     |████████████████████████████████| 74 kB 3.8 MB/s 


In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class Wikipediascraper(scrapy.Spider):
    # Name of your spider
    name = "cities"

    # Url to start your spider from 
    start_urls = [
        'https://en.wikipedia.org/wiki/List_of_largest_cities',
    ]

    # Callback function that will be called when starting your spider
    # It will get text, author and tags of the first <div> with class="quote"
    def parse(self, response):
        cities = response.xpath('//table[2]/tbody/tr/td[1]')

        for city in cities:

          yield {'city' : city.xpath('./a/text()').get(),
                 }


        

In [3]:
# Name of the file where the results will be saved
filename = "cities.json"

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    "FEEDS": {
        filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(Wikipediascraper)
process.start()

2022-02-21 15:48:44 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2022-02-21 15:48:44 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.1.0, Python 3.7.12 (default, Jan 15 2022, 18:48:18) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1m  14 Dec 2021), cryptography 36.0.1, Platform Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
2022-02-21 15:48:44 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-02-21 15:48:44 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-02-21 15:48:44 [scrapy.extensions.telnet] INFO: Telnet Password: 35873933f2edf70d
2022-02-21 15:48:44 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.l

**Read the json file with results from the crawling**

In [8]:
import pandas as pd
df = pd.read_json('/content/cities.json')
df

,city
0,Tokyo
1,Delhi
2,Shanghai
3,São Paulo
4,Mexico City
...,...
76,"Washington, D.C."
77,Yangon
78,Alexandria
79,Jinan


In [58]:
import requests
all_scores = pd.DataFrame()
for city in df.city.values:
  try:
    base_info = requests.get('https://api.teleport.org/api/cities/?search=' + city).json()
    city_url_id = base_info["_embedded"]["city:search-results"][0]['_links']['city:item']['href'] #on choppe le geoname_id
    scores_url = requests.get(city_url_id).json()['_links']['city:urban_area']['href'] + 'scores' #on reconstruit l'url des scores
    scores = requests.get(scores_url).json()['categories']
    current_scores = pd.DataFrame(scores)[['name', 'score_out_of_10']].set_index('name').T

  
  except :
    current_scores = pd.DataFrame()
  current_scores

  current_scores['city'] = city
#   current_scores = current_scores.T
  all_scores = pd.concat([all_scores, current_scores], axis=0)
all_scores



2022-02-21 16:58:45 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): api.teleport.org:443
2022-02-21 16:58:45 [urllib3.connectionpool] DEBUG: https://api.teleport.org:443 "GET /api/cities/?search=Tokyo HTTP/1.1" 200 692
2022-02-21 16:58:45 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): api.teleport.org:443
2022-02-21 16:58:45 [urllib3.connectionpool] DEBUG: https://api.teleport.org:443 "GET /api/cities/geonameid:1850147/ HTTP/1.1" 200 470
2022-02-21 16:58:45 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): api.teleport.org:443
2022-02-21 16:58:45 [urllib3.connectionpool] DEBUG: https://api.teleport.org:443 "GET /api/urban_areas/slug:tokyo/scores HTTP/1.1" 200 1046
2022-02-21 16:58:45 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): api.teleport.org:443
2022-02-21 16:58:46 [urllib3.connectionpool] DEBUG: https://api.teleport.org:443 "GET /api/cities/?search=Delhi HTTP/1.1" 200 644
2022-02-21 16:58:46 [urllib3.conn

,Housing,Cost of Living,Startups,Venture Capital,Travel Connectivity,Commute,Business Freedom,Safety,Healthcare,Education,Environmental Quality,Economy,Taxation,Internet Access,Leisure & Culture,Tolerance,Outdoors,city
score_out_of_10,5.6570,3.431,6.2970,3.963,8.1515,6.00750,8.377667,10.0000,9.253333,6.9805,5.95575,3.7235,6.0330,4.8350,9.3420,7.1945,6.5225,Tokyo
score_out_of_10,9.7520,9.869,9.8285,9.099,5.6420,3.71225,3.618000,6.4185,2.256333,0.5120,2.20300,5.5565,3.4145,3.5880,5.6830,3.1160,1.1785,Delhi
score_out_of_10,5.7440,7.179,5.7255,9.288,6.8620,5.08450,3.895667,7.2085,5.145000,5.6995,2.83875,6.1030,1.4960,2.6360,6.6935,2.7410,5.7175,Shanghai
score_out_of_10,8.3160,7.823,6.6225,4.253,2.2655,4.41200,2.781000,5.2955,5.462333,1.1040,2.65350,2.1870,2.8525,3.3895,8.7895,6.8955,7.0775,São Paulo
score_out_of_10,8.3600,9.244,7.4800,2.634,2.8700,3.80600,5.761000,5.4640,5.789667,1.7130,2.50625,3.5005,4.3335,5.8405,9.9620,7.0890,3.9140,Mexico City
score_out_of_10,10.0000,10.000,5.0985,2.958,3.7575,3.99025,5.790667,6.7910,2.897333,0.5000,1.15550,2.9780,5.1010,2.4050,6.1855,2.6500,0.5000,Cairo
score_out_of_10,8.1240,9.825,9.8530,6.179,5.4140,3.94000,3.618000,7.6185,2.268333,0.5275,2.09650,5.5565,3.4145,3.4290,4.9685,3.8390,5.7920,Mumbai
score_out_of_10,7.8315,4.819,2.6680,2.144,6.2015,5.25500,8.417667,10.0000,9.178000,5.5895,5.32675,3.7235,6.0330,3.7240,7.2265,7.1895,5.1875,Osaka
score_out_of_10,1.0000,2.342,10.0000,10.000,6.6750,5.51925,8.671000,7.0220,8.501667,8.0935,5.23375,6.5145,3.9205,7.0985,10.0000,6.7125,5.7475,New York
score_out_of_10,8.8120,7.764,4.6730,2.310,3.7700,4.92350,4.045333,6.0395,6.106667,1.4220,4.86900,2.8840,1.5680,1.5275,7.6735,8.5840,4.5355,Buenos Aires


In [59]:
all_scores.to_csv('all_cities_scores.csv')

In [65]:
s3.Bucket('currentbuckettoday').upload_file('/content/all_cities_scores.csv', Key = 'all_cities_scores.csv')

2022-02-21 17:05:39 [boto3.resources.factory] DEBUG: Loading s3:Bucket
2022-02-21 17:05:39 [boto3.resources.model] DEBUG: Renaming Bucket attribute name
2022-02-21 17:05:39 [botocore.hooks] DEBUG: Event creating-resource-class.s3.Bucket: calling handler <function lazy_call.<locals>._handler at 0x7fda87beab00>
2022-02-21 17:05:39 [s3transfer.utils] DEBUG: Acquiring 0
2022-02-21 17:05:39 [s3transfer.tasks] DEBUG: UploadSubmissionTask(transfer_id=0, {'transfer_future': <s3transfer.futures.TransferFuture object at 0x7fda887d2b10>}) about to wait for the following futures []
2022-02-21 17:05:39 [s3transfer.tasks] DEBUG: UploadSubmissionTask(transfer_id=0, {'transfer_future': <s3transfer.futures.TransferFuture object at 0x7fda887d2b10>}) done waiting for dependent futures
2022-02-21 17:05:39 [s3transfer.tasks] DEBUG: Executing task UploadSubmissionTask(transfer_id=0, {'transfer_future': <s3transfer.futures.TransferFuture object at 0x7fda887d2b10>}) with kwargs {'client': <botocore.client.S3 